## Lesson: Toy Differential Privacy - Simple Database Queries

In this section we're going to play around with Differential Privacy in the context of a database query. The database is going to be a VERY simple database with only one boolean column. Each row corresponds to a person. Each value corresponds to whether or not that person has a certain private attribute (such as whether they have a certain disease, or whether they are above/below a certain age). We are then going to learn how to know whether a database query over such a small database is differentially private or not - and more importantly - what techniques are at our disposal to ensure various levels of privacy


### First We Create a Simple Database

Step one is to create our database - we're going to do this by initializing a random list of 1s and 0s (which are the entries in our database). Note - the number of entries directly corresponds to the number of people in our database.

In [8]:
import torch

# the number of entries in our database
num_entries = 5000

db = torch.rand(num_entries) > 0.5
db

tensor([1, 1, 1,  ..., 1, 0, 0], dtype=torch.uint8)

## Project: Generate Parallel Databases

Key to the definition of differenital privacy is the ability to ask the question "When querying a database, if I removed someone from the database, would the output of the query be any different?". Thus, in order to check this, we must construct what we term "parallel databases" which are simply databases with one entry removed. 

In this first project, I want you to create a list of every parallel database to the one currently contained in the "db" variable. Then, I want you to create a function which both:

- creates the initial database (db)
- creates all parallel databases

In [9]:
# try project here!
db = torch.rand(num_entries) > 0.5
db

tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.uint8)

In [10]:
db[0:10]

tensor([1, 1, 1, 1, 1, 1, 0, 1, 1, 1], dtype=torch.uint8)

In [11]:
def get_parallel_db(db, remove_index):
    return torch.cat((db[0:remove_index], 
                      db[remove_index+1:]))

In [104]:
get_parallel_db(db, 3).shape

torch.Size([4999])

In [12]:
def get_parallel_dbs(db):
    
    parallel_dbs = list()
    
    for i in range(len(db)):
        pdb = get_parallel_db(db, i)
        parallel_dbs.append(pdb)

    return parallel_dbs

In [13]:
pdbs = get_parallel_dbs(db)
pdbs

[tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.uint8),
 tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.uint8),
 tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.uint8),
 tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.uint8),
 tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.uint8),
 tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.uint8),
 tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.uint8),
 tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.uint8),
 tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.uint8),
 tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.uint8),
 tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.uint8),
 tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.uint8),
 tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.uint8),
 tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.uint8),
 tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.uint8),
 tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.uint8),
 tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.uint8),
 tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.uint8),
 tensor([1, 1, 1,  ..., 0, 0

In [14]:
def create_db_and_parallels(num_entries):
    
    db = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(db)
    
    return db, pdbs

In [15]:
db, pdbs = create_db_and_parallels(20)
db

tensor([0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1],
       dtype=torch.uint8)

In [120]:
pdbs

[tensor([0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0,

In [119]:
len(pdbs)

20

# Lesson: Towards Evaluating The Differential Privacy of a Function

Intuitively, we want to be able to query our database and evaluate whether or not the result of the query is leaking "private" information. As mentioned previously, this is about evaluating whether the output of a query changes when we remove someone from the database. Specifically, we want to evaluate the *maximum* amount the query changes when someone is removed (maximum over all possible people who could be removed). So, in order to evaluate how much privacy is leaked, we're going to iterate over each person in the database and measure the difference in the output of the query relative to when we query the entire database. 

Just for the sake of argument, let's make our first "database query" a simple sum. Aka, we're going to count the number of 1s in the database.

In [165]:
db, pdbs = create_db_and_parallels(5000)

In [166]:
def query(db):
    return db.sum()

In [167]:
full_db_result = query(db)

In [168]:
sensitivity = 0
for pdb in pdbs:
    pdb_result = query(pdb)
    
    db_distance = torch.abs(pdb_result - full_db_result)
    
    if(db_distance > sensitivity):
        sensitivity = db_distance


In [169]:
sensitivity

tensor(1)

# Project - Evaluating the Privacy of a Function

In the last section, we measured the difference between each parallel db's query result and the query result for the entire database and then calculated the max value (which was 1). This value is called "sensitivity", and it corresponds to the function we chose for the query. Namely, the "sum" query will always have a sensitivity of exactly 1. However, we can also calculate sensitivity for other functions as well.

Let's try to calculate sensitivity for the "mean" function.

In [175]:
# try this project here!
db, pdbs = create_db_and_parallels(20)

In [176]:
def query_mean(db):
    return db.float().mean()

In [177]:
full_db_result = query_mean(db)

In [178]:
sensitivity = 0
for pdb in pdbs:
    pdb_result = query_mean(pdb)
    
    db_distance = torch.abs(pdb_result - full_db_result)
    
    if(db_distance > sensitivity):
        sensitivity = db_distance


In [179]:
sensitivity

tensor(0.0289)

In [180]:
def sensitivity(query, n_entries):
    
    db, pdbs = create_db_and_parallels(n_entries)
    
    full_db_result = query(db)
    
    max_distance = 0
    for pdb in pdbs:
        pdb_result = query(pdb)

        db_distance = torch.abs(pdb_result - full_db_result)

        if(db_distance > max_distance):
            max_distance = db_distance
            
    return max_distance

In [195]:
sensitivity(query, 20), sensitivity(query_mean, 20)

(tensor(1), tensor(0.0316))

In [ ]:
# DONE! :D

Wow! That sensitivity is WAY lower. Note the intuition here. "Sensitivity" is measuring how sensitive the output of the query is to a person being removed from the database. For a simple sum, this is always 1, but for the mean, removing a person is going to change the result of the query by rougly 1 divided by the size of the database (which is much smaller). Thus, "mean" is a VASTLY less "sensitive" function (query) than SUM.

# Project: Calculate L1 Sensitivity For Threshold

In this first project, I want you to calculate the sensitivty for the "threshold" function. 

- First compute the sum over the database (i.e. sum(db)) and return whether that sum is greater than a certain threshold.
- Then, I want you to create databases of size 10 and threshold of 5 and calculate the sensitivity of the function. 
- Finally, re-initialize the database 10 times and calculate the sensitivity each time.

In [251]:
# try this project here!
def query(db, threshold):
    return (db.sum() > threshold).float()

In [252]:
def sensitivity(query, n_entries, threshold):
    
    db, pdb = create_db_and_parallels(n_entries)
    
    full_db_result = query(db, threshold)

    max_distance = 0
    for pdb in pdbs:
        pdb_result = query(pdb, threshold)

        db_distance = torch.abs(pdb_result - full_db_result)

        if(db_distance > max_distance):
            max_distance = db_distance

    return max_distance

In [253]:
sensitivity(query, 10, 5)

tensor(1.)

In [254]:
def reinitialize_db(n_reinitialize, query, n_entries, threshold):
    for i in range(10):
        print(sensitivity(query, n_entries, threshold))

In [258]:
reinitialize_db(10, query, 10, 5)

tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
0
tensor(1.)
0
0


# Lesson: A Basic Differencing Attack

Sadly none of the functions we've looked at so far are differentially private (despite them having varying levels of sensitivity). The most basic type of attack can be done as follows.

Let's say we wanted to figure out a specific person's value in the database. All we would have to do is query for the sum of the entire database and then the sum of the entire database without that person!

# Project: Perform a Differencing Attack on Row 10

In this project, I want you to construct a database and then demonstrate how you can use two different sum queries to explose the value of the person represented by row 10 in the database (note, you'll need to use a database with at least 10 rows)

In [259]:
# try this project here!
db, _ = create_db_and_parallels(100)

In [260]:
pdb = get_parallel_db(db, remove_index=2)

In [261]:
db[2]

tensor(1, dtype=torch.uint8)

In [270]:
sum(db)

tensor(57, dtype=torch.uint8)

In [272]:
sum(pdb)

tensor(56, dtype=torch.uint8)

In [262]:
# differencing attack using sum query
sum(db) - sum(pdb)

tensor(1, dtype=torch.uint8)

In [265]:
# differencing attack using mean query
(sum(db).float()/len(db)) - (sum(pdb).float()/len(pdb))

tensor(0.0043)

In [271]:
# differencing attack using threshold query

(sum(db).float() > 56) - (sum(pdb).float() > 56)

tensor(1, dtype=torch.uint8)

# Project: Local Differential Privacy

As you can see, the basic sum query is not differentially private at all! In truth, differential privacy always requires a form of randomness added to the query. Let me show you what I mean.

### Randomized Response (Local Differential Privacy)

Let's say I have a group of people I wish to survey about a very taboo behavior which I think they will lie about (say, I want to know if they have ever committed a certain kind of crime). I'm not a policeman, I'm just trying to collect statistics to understand the higher level trend in society. So, how do we do this? One technique is to add randomness to each person's response by giving each person the following instructions (assuming I'm asking a simple yes/no question):

- Flip a coin 2 times.
- If the first coin flip is heads, answer honestly
- If the first coin flip is tails, answer according to the second coin flip (heads for yes, tails for no)!

Thus, each person is now protected with "plausible deniability". If they answer "Yes" to the question "have you committed X crime?", then it might becasue they actually did, or it might be becasue they are answering according to a random coin flip. Each person has a high degree of protection. Furthermore, we can recover the underlying statistics with some accuracy, as the "true statistics" are simply averaged with a 50% probability. Thus, if we collect a bunch of samples and it turns out that 60% of people answer yes, then we know that the TRUE distribution is actually centered around 70%, because 70% averaged wtih 50% (a coin flip) is 60% which is the result we obtained. 

However, it should be noted that, especially when we only have a few samples, the this comes at the cost of accuracy. This tradeoff exists across all of Differential Privacy. The greater the privacy protection (plausible deniability) the less accurate the results. 

Let's implement this local DP for our database before!

In [478]:
# try this project here!
'''
def flip_coin():    
    coin = torch.rand(2) > 0.5 # 1 for heads and 0 for tails
    return coin.float()
'''

In [479]:
'''
def decide_db(n_entries):
    
    db, _ = create_db_and_parallels(n_entries)
    print(db)
    
    for entry in db:
        coin = flip_coin()
        print("coins:" , coin[0], coin[1])
        if coin[0] == 1:
            print("coin1:" , coin[1])
            db[entry].float() * coin[0]
            #db[entry,0] = coin[1]
        else: 
            print("entry" , entry)
            #db[entry==entry] = entry
    return db
'''

In [512]:
db, pdbs = create_db_and_parallels(100)

In [523]:
# actual values from people (70% would be true)
def query(db):
    
    true_result = torch.mean(db.float())
    
    first_coin_flip = (torch.rand(len(db)) > 0.5).float()
    second_coin_flip = (torch.rand(len(db)) > 0.5).float()

    augmented_database = db.float() * first_coin_flip + (1 - first_coin_flip) * second_coin_flip

    db_result = torch.mean(augmented_database.float()) * 2 - 0.5
    
    return db_result, true_result

In [535]:
db, pdbs = create_db_and_parallels(10)
private_result, true_result = query(db)
print("With Noise:" + str(private_result))
print("Without Noise:" + str(true_result))

With Noise:tensor(0.3000)
Without Noise:tensor(0.6000)


In [536]:
db, pdbs = create_db_and_parallels(100)
private_result, true_result = query(db)
print("With Noise:" + str(private_result))
print("Without Noise:" + str(true_result))

With Noise:tensor(0.4200)
Without Noise:tensor(0.5000)


In [537]:
db, pdbs = create_db_and_parallels(1000)
private_result, true_result = query(db)
print("With Noise:" + str(private_result))
print("Without Noise:" + str(true_result))

With Noise:tensor(0.5080)
Without Noise:tensor(0.4710)


In [538]:
db, pdbs = create_db_and_parallels(10000)
private_result, true_result = query(db)
print("With Noise:" + str(private_result))
print("Without Noise:" + str(true_result))

With Noise:tensor(0.4980)
Without Noise:tensor(0.4909)


# Project: Varying Amounts of Noise

In this project, I want you to augment the randomized response query (the one we just wrote) to allow for varying amounts of randomness to be added. Specifically, I want you to bias the coin flip to be higher or lower and then run the same experiment. 

Note - this one is a bit tricker than you might expect. You need to both adjust the likelihood of the first coin flip AND the de-skewing at the end (where we create the "augmented_result" variable).

In [18]:
# try this project here! 
def query(db, noise):
    
    true_result = torch.mean(db.float())
    
    first_coin_flip = (torch.rand(len(db)) > noise).float()
    second_coin_flip = (torch.rand(len(db)) > 0.5).float()

    # Reskewing the dataset to have the same mean as the original
    augmented_database = db.float() * first_coin_flip + (1 - first_coin_flip) * second_coin_flip
    
    sk_result = augmented_database.float().mean()
    
    private_result = ((sk_result / noise) - 0.5) * noise / (1 - noise)
    
    return private_result, true_result

In [20]:
db, pdbs = create_db_and_parallels(10)
private_result, true_result = query(db, 0.2)
private_result_, true_result_ = query(db, 0.8)
print("With l_Noise:" + str(private_result))
print("Without l_Noise:" + str(true_result))
print("With h_Noise:" + str(private_result_))
print("Without h_Noise:" + str(true_result_))

With l_Noise:tensor(0.8750)
Without l_Noise:tensor(0.8000)
With h_Noise:tensor(0.)
Without h_Noise:tensor(0.8000)


In [21]:
db, pdbs = create_db_and_parallels(100)
private_result, true_result = query(db, 0.2)
private_result_, true_result_ = query(db, 0.8)
print("With l_Noise:" + str(private_result))
print("Without l_Noise:" + str(true_result))
print("With h_Noise:" + str(private_result_))
print("Without h_Noise:" + str(true_result_))

With l_Noise:tensor(0.5625)
Without l_Noise:tensor(0.5500)
With h_Noise:tensor(0.6000)
Without h_Noise:tensor(0.5500)


In [22]:
db, pdbs = create_db_and_parallels(1000)
private_result, true_result = query(db, 0.2)
private_result_, true_result_ = query(db, 0.8)
print("With l_Noise:" + str(private_result))
print("Without l_Noise:" + str(true_result))
print("With h_Noise:" + str(private_result_))
print("Without h_Noise:" + str(true_result_))

With l_Noise:tensor(0.4737)
Without l_Noise:tensor(0.4810)
With h_Noise:tensor(0.4700)
Without h_Noise:tensor(0.4810)


In [25]:
db, pdbs = create_db_and_parallels(10000)
private_result, true_result = query(db, 0.2)
print("With l_Noise:" + str(private_result))
print("Without l_Noise:" + str(true_result))


With l_Noise:tensor(0.5076)
Without l_Noise:tensor(0.5012)


In [26]:
db, pdbs = create_db_and_parallels(10000)
private_result_, true_result_ = query(db, 0.8)
print("With h_Noise:" + str(private_result_))
print("Without h_Noise:" + str(true_result_))

With h_Noise:tensor(0.5015)
Without h_Noise:tensor(0.4991)


# Lesson: The Formal Definition of Differential Privacy

The previous method of adding noise was called "Local Differentail Privacy" because we added noise to each datapoint individually. This is necessary for some situations wherein the data is SO sensitive that individuals do not trust noise to be added later. However, it comes at a very high cost in terms of accuracy. 

However, alternatively we can add noise AFTER data has been aggregated by a function. This kind of noise can allow for similar levels of protection with a lower affect on accuracy. However, participants must be able to trust that no-one looked at their datapoints _before_ the aggregation took place. In some situations this works out well, in others (such as an individual hand-surveying a group of people), this is less realistic.

Nevertheless, global differential privacy is incredibly important because it allows us to perform differential privacy on smaller groups of individuals with lower amounts of noise. Let's revisit our sum functions.

In [27]:
db, pdbs = create_db_and_parallels(100)

def query(db):
    return torch.sum(db.float())

def M(db):
    query(db) + noise

query(db)

tensor(47.)

So the idea here is that we want to add noise to the output of our function. We actually have two different kinds of noise we can add - Laplacian Noise or Gaussian Noise. However, before we do so at this point we need to dive into the formal definition of Differential Privacy.

![alt text](dp_formula.png "Title")

_Image From: "The Algorithmic Foundations of Differential Privacy" - Cynthia Dwork and Aaron Roth - https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf_

## This definition does not _create_ differential privacy, instead it is a measure of how much privacy is afforded by a query M. Specifically, it's a comparison between running the query M on a database (x) and a parallel database (y). As you remember, parallel databases are defined to be the same as a full database (x) with one entry/person removed.

Thus, this definition says that FOR ALL parallel databases, the maximum distance between a query on database (x) and the same query on database (y) will be e^epsilon, but that occasionally this constraint won't hold with probability delta. Thus, this theorem is called "epsilon delta" differential privacy.

# Epsilon

Let's unpack the intuition of this for a moment. 

Epsilon Zero: If a query satisfied this inequality where epsilon was set to 0, then that would mean that the query for all parallel databases outputed the exact same value as the full database. As you may remember, when we calculated the "threshold" function, often the Sensitivity was 0. In that case, the epsilon also happened to be zero.

Epsilon One: If a query satisfied this inequality with epsilon 1, then the maximum distance between all queries would be 1 - or more precisely - the maximum distance between the two random distributions M(x) and M(y) is 1 (because all these queries have some amount of randomness in them, just like we observed in the last section).

# Delta

Delta is basically the probability that epsilon breaks. Namely, sometimes the epsilon is different for some queries than it is for others. For example, you may remember when we were calculating the sensitivity of threshold, most of the time sensitivity was 0 but sometimes it was 1. Thus, we could calculate this as "epsilon zero but non-zero delta" which would say that epsilon is perfect except for some probability of the time when it's arbitrarily higher. Note that this expression doesn't represent the full tradeoff between epsilon and delta.

# Lesson: How To Add Noise for Global Differential Privacy

In this lesson, we're going to learn about how to take a query and add varying amounts of noise so that it satisfies a certain degree of differential privacy. In particular, we're going to leave behind the Local Differential privacy previously discussed and instead opt to focus on Global differential privacy. 

So, to sum up, this lesson is about adding noise to the output of our query so that it satisfies a certain epsilon-delta differential privacy threshold.

There are two kinds of noise we can add - Gaussian Noise or Laplacian Noise. Generally speaking Laplacian is better, but both are still valid. Now to the hard question...

### How much noise should we add?

The amount of noise necessary to add to the output of a query is a function of four things:

- the type of noise (Gaussian/Laplacian)
- the sensitivity of the query/function
- the desired epsilon (ε)
- the desired delta (δ)

Thus, for each type of noise we're adding, we have different way of calculating how much to add as a function of sensitivity, epsilon, and delta. We're going to focus on Laplacian noise. Laplacian noise is increased/decreased according to a "scale" parameter b. We choose "b" based on the following formula.

b = sensitivity(query) / epsilon

In other words, if we set b to be this value, then we know that we will have a privacy leakage of <= epsilon. Furthermore, the nice thing about Laplace is that it guarantees this with delta == 0. There are some tunings where we can have very low epsilon where delta is non-zero, but we'll ignore them for now.

### Querying Repeatedly

- if we query the database multiple times - we can simply add the epsilons (Even if we change the amount of noise and their epsilons are not the same).

# Project: Create a Differentially Private Query

In this project, I want you to take what you learned in the previous lesson and create a query function which sums over the database and adds just the right amount of noise such that it satisfies an epsilon constraint. Write a query for both "sum" and for "mean". Ensure that you use the correct sensitivity measures for both.

In [82]:
# try this project here!
import numpy as np

In [83]:
epsilon = 0.5 

In [84]:
db, pdbs = create_db_and_parallels(100)

In [86]:
def sum_query(db):
    return db.sum()

In [87]:
def laplacian_mechanism(db, query, sensitivity):
    
    beta = sensitivity / epsilon
    noise = torch.tensor(np.random.laplace(0, beta, 1))

    return query(db) + noise

In [94]:
def mean_query(db):
    return torch.mean(db.float())

In [97]:
mean_query(db)

tensor(0.5200)

In [93]:
laplacian_mechanism(db, sum_query, 1)

tensor([52.0219], dtype=torch.float64)

In [101]:
laplacian_mechanism(db, mean_query, 1/100) 

tensor([0.4745], dtype=torch.float64)

# Lesson: Differential Privacy for Deep Learning

So in the last lessons you may have been wondering - what does all of this have to do with Deep Learning? Well, these same techniques we were just studying form the core primitives for how Differential Privacy provides guarantees in the context of Deep Learning. 

Previously, we defined perfect privacy as "a query to a database returns the same value even if we remove any person from the database", and used this intuition in the description of epsilon/delta. In the context of deep learning we have a similar standard.

Training a model on a dataset should return the same model even if we remove any person from the dataset.

Thus, we've replaced "querying a database" with "training a model on a dataset". In essence, the training process is a kind of query. However, one should note that this adds two points of complexity which database queries did not have:

    1. do we always know where "people" are referenced in the dataset?
    2. neural models rarely never train to the same output model, even on identical data

The answer to (1) is to treat each training example as a single, separate person. Strictly speaking, this is often overly zealous as some training examples have no relevance to people and others may have multiple/partial (consider an image with multiple people contained within it). Thus, localizing exactly where "people" are referenced, and thus how much your model would change if people were removed, is challenging.

The answer to (2) is also an open problem - but several interesitng proposals have been made. We're going to focus on one of the most popular proposals, PATE.

## An Example Scenario: A Health Neural Network

First we're going to consider a scenario - you work for a hospital and you have a large collection of images about your patients. However, you don't know what's in them. You would like to use these images to develop a neural network which can automatically classify them, however since your images aren't labeled, they aren't sufficient to train a classifier. 

However, being a cunning strategist, you realize that you can reach out to 10 partner hospitals which DO have annotated data. It is your hope to train your new classifier on their datasets so that you can automatically label your own. While these hospitals are interested in helping, they have privacy concerns regarding information about their patients. Thus, you will use the following technique to train a classifier which protects the privacy of patients in the other hospitals.

- 1) You'll ask each of the 10 hospitals to train a model on their own datasets (All of which have the same kinds of labels)
- 2) You'll then use each of the 10 partner models to predict on your local dataset, generating 10 labels for each of your datapoints
- 3) Then, for each local data point (now with 10 labels), you will perform a DP query to generate the final true label. This query is a "max" function, where "max" is the most frequent label across the 10 labels. We will need to add laplacian noise to make this Differentially Private to a certain epsilon/delta constraint.
- 4) Finally, we will retrain a new model on our local dataset which now has labels. This will be our final "DP" model.

So, let's walk through these steps. I will assume you're already familiar with how to train/predict a deep neural network, so we'll skip steps 1 and 2 and work with example data. We'll focus instead on step 3, namely how to perform the DP query for each example using toy data.

So, let's say we have 10,000 training examples, and we've got 10 labels for each example (from our 10 "teacher models" which were trained directly on private data). Each label is chosen from a set of 10 possible labels (categories) for each image.

In [2]:
import numpy as np
import sys
np.set_printoptions(threshold=10)

In [3]:
num_teachers = 10 # we're working with 10 partner hospitals
num_examples = 10000 # the size of OUR dataset
num_labels = 10 # number of labels for our classifier

In [4]:
preds = (np.random.rand(num_teachers, num_examples) * num_labels).astype(int).transpose(1,0) # fake predictions

In [5]:
#preds[0].shape

In [6]:
#preds

In [7]:
#an_image = preds[:,0]

In [8]:
#an_image

In [9]:
#label_counts = np.bincount(an_image, minlength=num_labels)

In [10]:
'''
epsilon = 0.1 
beta = 1/ epsilon

for i in range(len(label_counts)):
    label_counts[i] += np.random.laplace(0, beta, 1)
'''

'\nepsilon = 0.1 \nbeta = 1/ epsilon\n\nfor i in range(len(label_counts)):\n    label_counts[i] += np.random.laplace(0, beta, 1)\n'

In [11]:
#label_counts

In [12]:
#np.argmax(label_counts)

In [13]:
new_labels = list()
for an_image in preds:

    label_counts = np.bincount(an_image, minlength=num_labels)

    epsilon = 0.1
    beta = 1 / epsilon

    for i in range(len(label_counts)):
        label_counts[i] += np.random.laplace(0, beta, 1)

    new_label = np.argmax(label_counts)
    
    new_labels.append(new_label)

In [14]:
# new labels
#len(new_labels)

In [15]:
new_labels

[5,
 0,
 2,
 7,
 8,
 6,
 9,
 0,
 7,
 1,
 2,
 0,
 4,
 0,
 0,
 5,
 3,
 8,
 4,
 9,
 4,
 8,
 3,
 2,
 1,
 0,
 2,
 6,
 6,
 8,
 9,
 4,
 8,
 2,
 7,
 5,
 7,
 7,
 4,
 3,
 4,
 8,
 8,
 8,
 2,
 0,
 4,
 5,
 8,
 5,
 6,
 6,
 6,
 4,
 0,
 7,
 2,
 7,
 3,
 9,
 3,
 2,
 9,
 1,
 8,
 4,
 9,
 8,
 7,
 8,
 3,
 4,
 6,
 0,
 3,
 8,
 4,
 6,
 5,
 4,
 6,
 5,
 8,
 3,
 5,
 8,
 3,
 2,
 9,
 9,
 2,
 0,
 3,
 7,
 5,
 2,
 3,
 5,
 9,
 9,
 6,
 5,
 7,
 4,
 4,
 3,
 1,
 2,
 0,
 5,
 1,
 3,
 8,
 6,
 5,
 5,
 0,
 8,
 3,
 6,
 6,
 2,
 1,
 0,
 3,
 8,
 1,
 0,
 5,
 4,
 9,
 5,
 5,
 1,
 8,
 7,
 7,
 2,
 1,
 4,
 3,
 4,
 5,
 5,
 3,
 2,
 9,
 0,
 1,
 8,
 9,
 0,
 6,
 4,
 3,
 3,
 1,
 2,
 5,
 9,
 2,
 0,
 1,
 9,
 8,
 5,
 0,
 9,
 8,
 8,
 8,
 2,
 0,
 4,
 4,
 8,
 6,
 0,
 5,
 3,
 3,
 1,
 6,
 0,
 2,
 3,
 0,
 3,
 1,
 3,
 2,
 7,
 6,
 4,
 0,
 2,
 9,
 8,
 0,
 3,
 7,
 4,
 5,
 9,
 1,
 1,
 7,
 1,
 7,
 0,
 4,
 1,
 1,
 2,
 8,
 1,
 2,
 2,
 1,
 6,
 5,
 4,
 7,
 2,
 0,
 8,
 3,
 2,
 7,
 4,
 0,
 8,
 5,
 8,
 6,
 3,
 5,
 9,
 7,
 6,
 1,
 8,
 1,
 8,
 7,
 9,
 2,
 8,
 5,
 6,


# PATE Analysis

In [16]:
labels = np.array([9, 9, 3, 6, 9, 9, 9, 9, 8, 2])
counts = np.bincount(labels, minlength=10)
query_result = np.argmax(counts)
query_result

9

In [19]:
from syft.frameworks.torch.differential_privacy import pate

In [20]:
num_teachers, num_examples, num_labels = (100, 100, 10)
preds = (np.random.rand(num_teachers, num_examples) * num_labels).astype(int) #fake preds
indices = (np.random.rand(num_examples) * num_labels).astype(int) # true answers

preds[:,0:10] *= 0

data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5)

assert data_dep_eps < data_ind_eps



In [21]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 11.756462732485115
Data Dependent Epsilon: 1.52655213289881


In [22]:
preds[:,0:50] *= 0

In [23]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5, moments=20)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 11.756462732485115
Data Dependent Epsilon: 0.9029013677789843


# Where to Go From Here


Read:
    - Algorithmic Foundations of Differential Privacy: https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf
    - Deep Learning with Differential Privacy: https://arxiv.org/pdf/1607.00133.pdf
    - The Ethical Algorithm: https://www.amazon.com/Ethical-Algorithm-Science-Socially-Design/dp/0190948205
   
Topics:
    - The Exponential Mechanism
    - The Moment's Accountant
    - Differentially Private Stochastic Gradient Descent

Advice:
    - For deployments - stick with public frameworks!
    - Join the Differential Privacy Community
    - Don't get ahead of yourself - DP is still in the early days

# Section Project:

For the final project for this section, you're going to train a DP model using this PATE method on the MNIST dataset, provided below.

###### In this first project, you will be given two things.

First, you'll be given a series of labelled private datasets which you must keep differentially private.

Second, you'll be given a public unlabeled dataset which you do not need to keep differentially private.

Your goal is to use the technique described in the last lesson to automatically label the second dataset.

You should then be able to train a model on this second public dataset and get a reasonable level of accuracy on the task given a certain epsilon delta constraint. 

Good luck in your project, and I'll see you in the next course.



In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import Subset

In [2]:
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5, ), (0.5,))])

train_data = datasets.MNIST(root='data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False, download=True, transform=transform)

In [3]:
len(test_data), len(train_data)

(10000, 60000)

In [4]:
batch_size = 32
num_teachers = 100

# to create data loaders for the teacher classifier
def data_loader(train_data, num_teachers):
    teacher_loaders = []
    data_size = len(train_data) // num_teachers # Number of records per hospital
    
    for i in range(num_teachers):
        indices = list(range(i * data_size, (i+1) * data_size))
        subset = Subset(train_data, indices)
        loader = torch.utils.data.DataLoader(subset, batch_size=batch_size)
        teacher_loaders.append(loader)
    return teacher_loaders

teacher_loaders = data_loader(train_data, num_teachers)


In [5]:
'''create public dataset by using 90% of the test data as train data 
and remaining 10% as test data'''

student_train_data = Subset(test_data, list(range(9000)))
student_test_data = Subset(test_data, list(range(9000, 10000)))

student_train_loader = torch.utils.data.DataLoader(student_train_data,
                                                   batch_size=batch_size)
student_test_loader = torch.utils.data.DataLoader(student_test_data, 
                                                  batch_size=batch_size)

In [13]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [14]:
def train(model, trainloader, criterion, optimizer, epochs=10):
    
    '''trains a SINGLE CLASSIFIER model'''
    
    running_loss = 0
    for e in range(epochs):
        model.train()
        
        for images, labels in trainloader:
            optimizer.zero_grad()
            
            output = model.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
def predict(model, dataloader):
    
    '''predicts labels for a dataset given the model 
    and dataloader as inputs'''

    outputs = torch.zeros(0, dtype=torch.long)
    model.eval()
    
    for images, labels in dataloader:
        output = model.forward(images)
        ps = torch.argmax(torch.exp(output), dim=1)
        outputs = torch.cat((outputs, ps))
    
    return outputs

def train_models(num_teachers):
    
    '''trains the *number of teacher* classifiers'''

    models = []
    for i in range(num_teachers):
        model = Classifier()
        criterion = nn.NLLLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.003)
        train(model, teacher_loaders[i], criterion, optimizer)
        models.append(model)
        
    return models

models = train_models(num_teachers)

/Users/ismailkorkmaz/anaconda3/envs/pysyft/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [17]:
import numpy as np

epsilon = 0.2 

def aggregated_teacher(models, dataloader, epsilon):
    
    '''Take predictions from individual teacher model and 
    creates the true labels for the student 
    after adding laplacian noise to them'''
    
    preds = torch.torch.zeros((len(models), 9000), dtype=torch.long)
    for i, model in enumerate(models):
        results = predict(model, dataloader)
        preds[i] = results 
        
    labels = np.array([]).astype(int)
    for image_preds in np.transpose(preds):
        label_counts = np.bincount(image_preds, minlength=10)
        beta = 1 / epsilon
        
        for i in range(len(label_counts)):
            label_counts[i] += np.random.laplace(0, beta, 1)
            
        new_label = np.argmax(label_counts)
        labels = np.append(labels, new_label)
        
    return preds.numpy(), labels

teacher_models = models
preds, student_labels = aggregated_teacher(teacher_models, 
                                           student_train_loader, epsilon)

/Users/ismailkorkmaz/anaconda3/envs/pysyft/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [18]:
def student_loader(student_train_loader, labels):
    for i, (data, _) in enumerate(iter(student_train_loader)):
        yield data, torch.from_numpy(labels[i*len(data):(i+1)*len(data)])
        
student_model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(student_model.parameters(), lr=0.003)
epochs = 10
steps = 0
running_loss = 0
for e in range(epochs):
    student_model.train()
    train_loader = student_loader(student_train_loader, student_labels)
    for images, labels in train_loader:
        steps += 1
        
        optimizer.zero_grad()
        output = student_model.forward(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % 50 == 0:
            test_loss = 0
            accuracy = 0
            student_model.eval()
            with torch.no_grad():
                for images, labels in student_test_loader:
                    log_ps = student_model(images)
                    test_loss += criterion(log_ps, labels).item()
                    
                    # Accuracy
                    ps = torch.exp(log_ps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor))
            student_model.train()
            print("Epoch: {}/{}.. ".format(e+1, epochs),
                 "Training Loss: {:.3f}.. ".format(running_loss/len(student_train_loader)),
                 "Test Loss: {:.3f}.. ".format(test_loss/len(student_test_loader)),
                 "Test Accuracy: {:.3f}".format(accuracy/len(student_test_loader)))
            running_loss = 0

/Users/ismailkorkmaz/anaconda3/envs/pysyft/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 1/10..  Training Loss: 0.329..  Test Loss: 0.946..  Test Accuracy: 0.729
Epoch: 1/10..  Training Loss: 0.171..  Test Loss: 0.518..  Test Accuracy: 0.878
Epoch: 1/10..  Training Loss: 0.130..  Test Loss: 0.363..  Test Accuracy: 0.905
Epoch: 1/10..  Training Loss: 0.097..  Test Loss: 0.340..  Test Accuracy: 0.909
Epoch: 1/10..  Training Loss: 0.092..  Test Loss: 0.294..  Test Accuracy: 0.918
Epoch: 2/10..  Training Loss: 0.122..  Test Loss: 0.271..  Test Accuracy: 0.930
Epoch: 2/10..  Training Loss: 0.095..  Test Loss: 0.254..  Test Accuracy: 0.927
Epoch: 2/10..  Training Loss: 0.079..  Test Loss: 0.228..  Test Accuracy: 0.938
Epoch: 2/10..  Training Loss: 0.074..  Test Loss: 0.198..  Test Accuracy: 0.940
Epoch: 2/10..  Training Loss: 0.061..  Test Loss: 0.254..  Test Accuracy: 0.926
Epoch: 2/10..  Training Loss: 0.060..  Test Loss: 0.221..  Test Accuracy: 0.938
Epoch: 3/10..  Training Loss: 0.115..  Test Loss: 0.239..  Test Accuracy: 0.938
Epoch: 3/10..  Training Loss: 0.080..  T

In [20]:
from syft.frameworks.torch.differential_privacy import pate

data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, 
                                                   indices=student_labels, 
                                                   noise_eps=epsilon, 
                                                   delta=1e-5)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 1451.5129254649705
Data Dependent Epsilon: 12.77557271368862
